In [ ]:
# Economic 

In [3]:
import numpy as np

In [9]:
def calculate_total_cost(welfare_program, population):
    total_cost = 0

    # Loop over the joint distribution
    for i in range(100):  # loop over wealth percentiles
        for j in range(100):  # loop over income percentiles
            average_income = population.income_percentiles[j]
            average_wealth = population.wealth_percentiles[i]

            # Compute benefit for an individual in this bin
            E = .5  # Eligibility set to constant .5 for now - we can change to a function of wealth and income percentiles
            individual_benefit = E * min(welfare_program.G + welfare_program.S * average_income, welfare_program.M, 
                                         max([welfare_program.M - welfare_program.T * (average_income - welfare_program.P), 0]))

            # Multiply the individual benefit by number of individuals in this bin to get total benefit for the bin
            total_benefit_for_bin = individual_benefit * population.number_of_individuals(i, j)

            # Calculate total cost of providing the benefit for this bin
            bin_cost = total_benefit_for_bin / welfare_program.efficiency

            # Accumulate the cost
            total_cost += bin_cost

    return total_cost

In [15]:
import numpy as np

# Classes for Individual, Firm, and Government
class Individual:
    def __init__(self, income, wealth=0):
        self.income = income
        self.wealth = wealth
        self.tax_paid = 0

class Firm:
    def __init__(self, net_income, wealth):
        self.net_income = net_income
        self.wealth = wealth
        self.tax_paid = 0

class Population:
    def __init__(self, total_count, wealth_percentiles, income_percentiles, joint_distribution):
        self.total_count = total_count
        self.wealth_percentiles = wealth_percentiles  # Array of length 100 for wealth values
        self.income_percentiles = income_percentiles  # Array of length 100 for income values
        self.joint_distribution = joint_distribution  # 100x100 array with joint distribution of wealth and income

    def number_of_individuals(self, wealth_index, income_index):
        """Returns the number of individuals for a specific wealth and income percentile."""
        percentage = self.joint_distribution[wealth_index][income_index]
        return self.total_count * percentage

class Government:
    def __init__(self):
        self.revenue = 0
        self.non_welfare_expenditure = 1000000  # Placeholder
        # Other government budget attricutes

    def total_expenditure(self, welfare_programs, population):
        welfare_expenditures = sum([calculate_total_cost(program, population) for program in welfare_programs])
        return self.non_welfare_expenditure + welfare_expenditures

class WelfareProgram:
    def __init__(self, G, S, M, P, T, efficiency, participation_rate):
        self.G = G
        self.S = S
        self.M = M
        self.P = P
        self.T = T
        self.efficiency = efficiency
        self.participation_rate = participation_rate


# Test with some example welfare programs
welfare_program1 = WelfareProgram(G=5000, S=0.1, M=10000, P=30000, T=0.05, efficiency=0.7, participation_rate=0.9)
welfare_program2 = WelfareProgram(G=3000, S=0.15, M=7000, P=25000, T=0.07, efficiency=0.8, participation_rate=1.0)
welfare_programs = [welfare_program1, welfare_program2]

In [16]:
# Global constant for poverty line
POVERTY_LINE = 12700  # Placeholder value, adjust as needed

# UBI policy definition
class UBIProgram(WelfareProgram):
    def __init__(self, alpha, efficiency=1.0):
        G = alpha * POVERTY_LINE
        S = 0  # No subsidy on earned income for UBI
        M = G  # Maximum allowable grant is the same as G for UBI
        P = float('inf')  # No phase-out for UBI
        T = 0  # No tax rate on income above phase-out for UBI
        participation_rate = 1.0  # 100% participation rate
        super().__init__(G, S, M, P, T, efficiency, participation_rate)

In [18]:
class ProgressiveTax:
    def __init__(self, brackets):
        """
        Initialize with a dictionary representing the tax brackets.

        :param brackets: Dictionary where `income: rate` represents the marginal tax rate beginning at `income` and
                         ending at the value of the next key.
        """
        # Sort the brackets based on income
        self.brackets = dict(sorted(brackets.items()))

    def compute_tax(self, income):
        """
        Compute the tax for a given income based on the progressive tax brackets.

        :param income: Income for which to compute the tax.
        :return: Computed tax.
        """
        tax = 0
        previous_bracket_limit = 0

        # Iterate through the sorted brackets
        for bracket_limit, rate in self.brackets.items():
            # If income is within the current bracket
            if income <= bracket_limit:
                tax += (income - previous_bracket_limit) * rate
                return tax
            else:
                # Compute tax for the entire bracket range and move to the next
                tax += (bracket_limit - previous_bracket_limit) * rate
                previous_bracket_limit = bracket_limit

        # If income is beyond the highest bracket, apply the highest rate to the remainder
        tax += (income - previous_bracket_limit) * rate
        return tax

"""
# Example: uncomment to run

tax_brackets = {
    10000: 0.1,   # 10% for income <= 10,000
    50000: 0.2,  # 20% for income between 10,001 and 50,000
    100000: 0.3  # 30% for income > 50,000
}
progressive_tax = ProgressiveTax(tax_brackets)
print(progressive_tax.compute_tax(20000))  # Example usage
"""

'\n# Example: uncomment to run\n\ntax_brackets = {\n    10000: 0.1,   # 10% for income <= 10,000\n    50000: 0.2,  # 20% for income between 10,001 and 50,000\n    100000: 0.3  # 30% for income > 50,000\n}\nprogressive_tax = ProgressiveTax(tax_brackets)\nprint(progressive_tax.compute_tax(20000))  # Example usage\n'

In [19]:
class TaxPolicy:
    def __init__(self, individual_tax_policy, corporate_tax_policy, other_tax_params):
        """
        Initialize the TaxPolicy with individual and corporate progressive tax policies.

        :param individual_tax_policy: A ProgressiveTax object for individual income tax.
        :param corporate_tax_policy: A ProgressiveTax object for corporate tax.
        :param other_tax_params: Parameters for other taxes. This can be expanded upon.
        """
        self.individual_tax_policy = individual_tax_policy
        self.corporate_tax_policy = corporate_tax_policy
        self.other_tax_params = other_tax_params

    def individual_income_tax(self, individual_income):
        """
        Compute individual income tax based on the given income and the tax policy.

        :param individual_income: The income for which to compute the tax.
        :return: Computed tax.
        """
        return self.individual_tax_policy.compute_tax(individual_income)

    def corporate_tax(self, corporate_income):
        """
        Compute corporate tax based on the given corporate income and the tax policy.

        :param corporate_income: The corporate income for which to compute the tax.
        :return: Computed tax.
        """
        return self.corporate_tax_policy.compute_tax(corporate_income)

    def other_taxes(self):
        # Compute other taxes.
        # Placeholder for now; this can be expanded upon based on the nature of other taxes.
        pass

    def total_individual_income_tax_revenue(self, population):
        """
        Compute the total revenue from individual income taxes based on the population's income distribution.

        :param population: Population object with joint distribution of income and wealth.
        :return: Total revenue from individual income taxes.
        """
        total_revenue = 0

        # Iterate over the income percentiles
        for j in range(100):
            average_income = population.income_percentiles[j]
            tax_for_income_bin = self.individual_income_tax(average_income)

            # For each wealth percentile, compute the number of individuals and add to the total revenue
            for i in range(100):
                total_revenue += tax_for_income_bin * population.number_of_individuals(i, j)

        return total_revenue

We now switch to examining individual consumer behavior: namely, preferences detailing their consumption demand and labor supply functions.

Homogeneous Agents: We'll use a representative agent model, where one agent's behavior is representative of the entire population.

Focus on Consumption: Given an income (which might include UBI), the representative agent decides how much to consume. This will be based on a simple utility function of consumption.

Fixed Prices: Prices of goods and services are constant. We'll note to reintroduce inflationary considerations in future iterations.

Simple Tax and UBI System: We will implement the UBI policy and tax system we've defined so far to see how the introduction of UBI affects the representative agent's income and thus consumption.

Now, let's proceed by building the model with these considerations:

In [21]:
class RepresentativeAgent:
    def __init__(self, initial_income):
        self.income = initial_income  # initial income without UBI
        self.consumption = 0

    def utility(self, consumption):
        """Utility function based on consumption."""
        # As a simple function, we can use the square root to represent diminishing returns.
        return np.sqrt(consumption)

    def decide_consumption(self):
        """Decide how much to consume given the current income."""
        # For now, let's assume the agent consumes all of their income.
        # This can be refined later with more sophisticated consumption functions.
        self.consumption = self.income

    def receive_ubi(self, ubi_amount):
        """Receive UBI and adjust income."""
        self.income += ubi_amount

    def pay_tax(self, tax_amount):
        """Pay taxes and adjust income."""
        self.income -= tax_amount

# Example
agent = RepresentativeAgent(50000)  # starting with an income of 50,000
ubi_amount = 12000  # Let's say UBI is 12,000 annually

# Introducing UBI
agent.receive_ubi(ubi_amount)

# Decide consumption post-UBI
agent.decide_consumption()
print(f"After receiving UBI, the agent consumes: ${agent.consumption}")

After receiving UBI, the agent consumes: $62000
